# Preprocessing

In [2]:
# Unzip files in their respective folders
from scripts.unzip_files import unzip_files_KGS 

source_folder = '../data/zip_files'
destination_folder = '../data/v1.0_raw_data'

unzip_files_KGS(source_folder, destination_folder)

In [1]:
# Download las files from KGS
from scripts.download_las_files import download_and_organize_las_files

source_folder = '../data/v1.0_raw_data'  
destination_folder = '../data/v2.0_zip_files'

# File to map las file names to urls
database_path = '../data/zip_files/ks_las_files.txt'  

# Download and organize las files
download_and_organize_las_files(source_folder, destination_folder, database_path)

JonesCanyonSoutheast |                                                  
Schaben              |                                                  
Marcotte             |                                                  
Cutter               |                                                  
MedicineLodgeNorth   |                                                  
KraftPrusa           |                                                  
Wellington           |                                                  
Burrton              |                                                  
GarnettShoestring    |                                                  
BemisShutts          |                                                  
BradshawGasArea      |                                                  
Webster              |                                                  
Morel                |                                                  
ElDorado             |                             

In [1]:
from scripts.ajust_las_files import process_las_files

source_folder = '../data/v2.0_zip_files'
destination_folder = '../data/v3.0_las_files'
csv_folder = '../data/v1.0_raw_data'

process_las_files(source_folder, destination_folder, csv_folder)

Arroyo               |                                                  
BemisShutts          |                                                  
BradshawGasArea      |                                                  
Burrton              |                                                  
Cooper               |                                                  
EastFork             |                                                  
Cutter               |                                                  
ElDorado             |                                                  
GarnettShoestring    |                                                  
HallGurney           |                                                  
JonesCanyonSoutheast |                                                  
KraftPrusa           |                                                  
Marcotte             |                                                  
MedicineLodgeNorth   |                             

### PENDIENTES:

LLM with RAG to check formation

In [3]:
%load_ext autoreload
%autoreload 2

from scripts.widgets import create_directory_selector
from scripts.exploratory_data_analysis import list_curves_from_field
import os

base_data_path = '../data/v3.0_las_files'
destination_data_path = '../data/v4.0_data_filtere'

# Create the source directory selector widget
source_directory_selector = create_directory_selector('Select Source Field:', base_data_path)
display(source_directory_selector)

# Global variable to store the curve information
curves_info = {}
project = None

# Function to execute when a source directory is selected
def on_source_directory_selected(change):
    global curves_info, project
    
    # Get the full path to the selected directory
    source_folder = os.path.join(base_data_path, change['new'])
    
    # Get the curves information from the selected field
    project, curves_info = list_curves_from_field(source_folder)

# Set up the widget to observe changes and execute the function
source_directory_selector.observe(on_source_directory_selected, names='value')

Dropdown(description='Select Source Field:', options=('JonesCanyonSoutheast', 'Schaben', 'Marcotte', 'Cutter',…

In [ ]:
project

Index,UWI,Data,Curves
0,,22 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, CNSS, CNDL, GR, MCAL, MI, MN, DT, ITT, SPOR"
1,,22 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, CNSS, CNDL, GR, MCAL, MI, MN, DT, ITT, SPOR"
2,Mary Ann No.1-1_2,20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"
3,,22 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, CNSS, CNDL, GR, MCAL, MI, MN, DT, ITT, SPOR"
4,,22 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, CNSS, CNDL, GR, MCAL, MI, MN, DT, ITT, SPOR"
5,Walter #3-1_2,20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"
6,,20 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, GR, MCAL, MI, MN, DT, ITT, SPOR"
7,,20 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, GR, MCAL, MI, MN, DT, ITT, SPOR"
8,,22 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, CNSS, CNDL, GR, MCAL, MI, MN, DT, ITT, SPOR"
9,Walter Bros # 1-6_4,20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"


In [ ]:
source = '../data/v2.0_zip_files'

# write a code to count all .las files in source
